# Week 12 Graded Project — CrewAI Multi-Agent Workflow  
## Use Case 4: SaaS Product Support – Technical Issue Diagnosis

This notebook implements a goal-oriented multi-agent workflow for SaaS technical support.

### Agents:
1. Technical Issue Diagnosis Agent  
2. Product Knowledge Reasoning Agent (Static, Non-RAG)  
3. Troubleshooting Response Agent  
4. Engineering Escalation Agent  

### Requirements Covered:
- 4 collaborating agents
- Structured JSON handoffs
- Deterministic escalation logic
- Sample test inputs (routine + high-risk cases)
- No RAG (static product documentation)

In [ ]:
import os
print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))

## Imports

In [ ]:
import json
import re
import time
from typing import List, Optional, Dict, Any

from pydantic import BaseModel, Field
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

## Static Product Knowledge (Non-RAG)

In [ ]:
PRODUCT_DOCS = """
SaaS Product Support Documentation (Static)

Login Issues:
- Reset password if login fails.
- Check 2FA if OTP not received.
- If repeated failures: check account lock.

API Errors:
- 401: Unauthorized (check API key)
- 403: Forbidden (permissions issue)
- 500: Server-side issue (possible outage)

Integration Failures:
- Check webhook configuration
- Validate endpoint URL
- Ensure correct authentication token

Escalation Conditions:
- Production outage
- Data loss risk
- Repeated 500 errors
- Customer reports system-wide failure
"""

## Data Models

In [ ]:
class DiagnosisOutput(BaseModel):
    issue_category: str
    severity: int = Field(..., ge=0, le=100)
    confidence: float = Field(..., ge=0, le=1)

class KnowledgeOutput(BaseModel):
    recommended_steps: List[str]
    technical_notes: str

class TroubleshootOutput(BaseModel):
    draft_response: str

class EscalationOutput(BaseModel):
    escalate: bool
    escalation_reason: str = ""

class Handoff(BaseModel):
    customer_query: str
    issue_category: Optional[str] = None
    severity: Optional[int] = None
    confidence: Optional[float] = None
    recommended_steps: List[str] = []
    technical_notes: Optional[str] = None
    draft_response: Optional[str] = None
    risk_score: Optional[int] = None
    escalate: bool = False
    escalation_reason: Optional[str] = None

## Agent Prompts

In [ ]:
DIAGNOSIS_PROMPT = """
You are a Technical Issue Diagnosis Agent.

Classify into one of:
- login_issue
- api_error
- integration_issue
- outage
- general_query

Return JSON:
issue_category, severity (0-100), confidence (0-1)

Customer Query:
{customer_query}
"""

KNOWLEDGE_PROMPT = """
You are a Product Knowledge Reasoning Agent.

Documentation:
{docs}

Inputs:
issue_category={issue_category}
severity={severity}

Return JSON:
recommended_steps (array)
technical_notes (string)
"""

TROUBLESHOOT_PROMPT = """
You are a Troubleshooting Response Agent.

Inputs:
customer_query={customer_query}
issue_category={issue_category}
recommended_steps={recommended_steps}

Return JSON:
draft_response (string)
"""

ESCALATION_PROMPT = """
You are an Engineering Escalation Agent.

Escalate if:
- severity >= 80
- outage detected
- repeated API 500 errors
- confidence < 0.6

Return JSON:
escalate (true/false)
escalation_reason (string)

Context:
issue_category={issue_category}
severity={severity}
confidence={confidence}
customer_query={customer_query}
"""

## LLM and Agents

In [ ]:
# =========================
# LLM + Agents (Use Case 4)
# =========================

from langchain_openai import ChatOpenAI
from crewai import Agent

# LLM (make sure OPENAI_API_KEY is already set in environment)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 1) Diagnosis Agent
diagnosis_agent = Agent(
    role="Diagnosis Agent",
    goal=(
        "Categorize the user's SaaS technical issue accurately and extract key troubleshooting context "
        "(symptoms, error messages, environment, steps already tried)."
    ),
    backstory=(
        "You are a senior L1/L2 technical support engineer for a SaaS product. "
        "You triage incoming tickets, identify the most likely category (login, API, integration, bug, "
        "performance, permissions, billing), and capture the minimum required details for resolution."
    ),
    llm=llm,
    verbose=False
)

# 2) Product Knowledge / Policy Agent (non-RAG)
knowledge_agent = Agent(
    role="Product Knowledge Agent",
    goal=(
        "Apply static product rules, known troubleshooting playbooks, and safe checks (non-RAG) "
        "to decide the best next steps and constraints."
    ),
    backstory=(
        "You are the product SME and internal documentation owner. "
        "You know common failure modes, configuration requirements, and safe troubleshooting steps. "
        "You NEVER request secrets (passwords, API keys, OTPs). You suggest secure alternatives."
    ),
    llm=llm,
    verbose=False
)

# 3) Troubleshooting Response Agent
troubleshoot_agent = Agent(
    role="Troubleshooting Response Agent",
    goal=(
        "Draft a clear, customer-friendly troubleshooting response with step-by-step instructions, "
        "expected outcomes, and what to do if a step fails."
    ),
    backstory=(
        "You write support responses that are easy to follow. "
        "You keep steps minimal, structured, and safe. "
        "You avoid risky actions like deleting data unless explicitly instructed by policy and confirmed."
    ),
    llm=llm,
    verbose=False
)

# 4) Engineering Escalation Agent
escalation_agent = Agent(
    role="Engineering Escalation Agent",
    goal=(
        "Decide whether the issue should be escalated to engineering based on severity, security risk, "
        "uncertainty/low confidence, repeated failures, or potential data loss."
    ),
    backstory=(
        "You are the on-call support lead. "
        "You escalate issues involving security incidents, suspected breaches, data loss risk, "
        "production outages, or when the diagnosis is uncertain and the customer is blocked."
    ),
    llm=llm,
    verbose=False
)

print("✅ LLM + Agents created successfully.")


## Utility Functions

In [ ]:
def parse_json(text):
    text = str(text)
    match = re.search(r"\{[\s\S]*\}", text)
    if not match:
        raise ValueError("No JSON found")
    return json.loads(match.group(0))

def run_agent(agent, prompt, expected_output):
    task = Task(description=prompt, expected_output=expected_output, agent=agent)
    crew = Crew(agents=[agent], tasks=[task], verbose=False)
    result = crew.kickoff(inputs={})
    return parse_json(result)

def risk_score(issue_category, severity, confidence):
    score = int(severity * 0.5)
    if confidence < 0.6:
        score += 30
    if issue_category == "outage":
        score += 40
    return min(score, 100)

## Orchestrator

In [ ]:
def run_saas_workflow(customer_query: str):
    diag_raw = run_agent(diagnosis_agent,
                         DIAGNOSIS_PROMPT.format(customer_query=customer_query),
                         "JSON with keys: issue_category, severity, confidence")
    diag = DiagnosisOutput(**diag_raw)

    know_raw = run_agent(knowledge_agent,
                         KNOWLEDGE_PROMPT.format(docs=PRODUCT_DOCS,
                                                 issue_category=diag.issue_category,
                                                 severity=diag.severity),
                         "JSON with keys: recommended_steps, technical_notes")
    know = KnowledgeOutput(**know_raw)

    resp_raw = run_agent(troubleshoot_agent,
                         TROUBLESHOOT_PROMPT.format(customer_query=customer_query,
                                                    issue_category=diag.issue_category,
                                                    recommended_steps=know.recommended_steps),
                         "JSON with key: draft_response")
    resp = TroubleshootOutput(**resp_raw)

    esc_raw = run_agent(escalation_agent,
                        ESCALATION_PROMPT.format(issue_category=diag.issue_category,
                                                 severity=diag.severity,
                                                 confidence=diag.confidence,
                                                 customer_query=customer_query),
                        "JSON with keys: escalate, escalation_reason")
    esc = EscalationOutput(**esc_raw)

    handoff = Handoff(
        customer_query=customer_query,
        issue_category=diag.issue_category,
        severity=diag.severity,
        confidence=diag.confidence,
        recommended_steps=know.recommended_steps,
        technical_notes=know.technical_notes,
        draft_response=resp.draft_response,
        escalate=esc.escalate,
        escalation_reason=esc.escalation_reason
    )

    handoff.risk_score = risk_score(diag.issue_category, diag.severity, diag.confidence)
    if handoff.risk_score >= 70:
        handoff.escalate = True
        if not handoff.escalation_reason:
            handoff.escalation_reason = "High risk score safeguard triggered."

    return handoff

## Quick Test

In [ ]:
query = "Our production API is returning 500 errors for all users."
result = run_saas_workflow(query)
print(result.model_dump_json(indent=2))

## FULL test Suite (with escallation Edge cases)

In [ ]:
# =========================
# FULL TEST SET (Use Case 4)
# =========================

TEST_QUERIES = [

    # ----------------------------
    # Routine / Non-escalation
    # ----------------------------
    "I am unable to log in to my account. It says incorrect password.",
    "My API call is returning 401 unauthorized error.",
    "Webhook integration is not triggering after setup.",
    "How do I reset my password?",
    
    # ----------------------------
    # Moderate severity
    # ----------------------------
    "Our API returns 403 forbidden even though the key is correct.",
    "Users are reporting slow response times in the dashboard.",
    
    # ----------------------------
    # High severity / Escalation cases
    # ----------------------------
    "Our production API is returning 500 errors for all users.",
    "We suspect data corruption after last deployment.",
    "System is completely down for all customers. This is urgent!",
    "I think someone accessed our account without authorization.",
    
    # ----------------------------
    # Low confidence / ambiguous
    # ----------------------------
    "Something seems wrong but I don't know what exactly is happening."
]

for q in TEST_QUERIES:
    print("\n" + "="*100)
    print("USER QUERY:", q)
    
    try:
        result = run_saas_workflow(q)
        print(result.model_dump_json(indent=2))
    except Exception as e:
        print("ERROR:", e)
